In [1]:
result_dir <- "/mnt/volume1/qyc/auto-annot/results/"
tools <- c("RF","scVI", "singleCellNet", "SingleR", "SVM")

# Load required libraries
library(ggplot2)
library(readr)
library(dplyr)

Warning message:
“package ‘ggplot2’ was built under R version 4.4.3”
Warning message:
“package ‘readr’ was built under R version 4.4.1”
Warning message:
“package ‘dplyr’ was built under R version 4.4.1”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
read_time_data <- function(tool, sample) {
  # Try to read total time first
  total_time_file <- file.path(result_dir, tool, sample, paste0(tool, "_Total_Time.csv"))
  if (file.exists(total_time_file)) {
    time_data <- read.csv(total_time_file, header = FALSE)
    # Skip the first row (0.0) and calculate mean and sd of the next 5 rows
    times <- as.numeric(time_data$V1[2:6])
    return(list(mean = mean(times), sd = sd(times)))
  }
  
  # If total time doesn't exist, calculate from train + test
  train_time_file <- file.path(result_dir, tool, sample, paste0(tool, "_Training_Time.csv"))
  test_time_file <- file.path(result_dir, tool, sample, paste0(tool, "_Testing_Time.csv"))
  
  if (file.exists(train_time_file) && file.exists(test_time_file)) {
    train_times <- as.numeric(read.csv(train_time_file, header = FALSE)$V1[2:6])
    test_times <- as.numeric(read.csv(test_time_file, header = FALSE)$V1[2:6])
    total_times <- train_times + test_times
    return(list(mean = mean(total_times), sd = sd(total_times)))
  }
  
  return(list(mean = NA, sd = NA))
}

In [3]:
get_cell_number <- function(sample) {
  data_paths <- list(
    "10Xv2_pbmc1" = "/mnt/volume1/qyc/data/Inter-dataset/PbmcBench/10Xv2/10Xv2_pbmc1.csv",
    "10Xv3_pbmc1" = "/mnt/volume1/qyc/data/Inter-dataset/PbmcBench/10Xv3/10Xv3_pbmc1.csv",
    "CL_pbmc1" = "/mnt/volume1/qyc/data/Inter-dataset/PbmcBench/CEL-Seq/CL_pbmc1.csv",
    "10x_5cl_data" = "/mnt/volume1/qyc/data/Intra-dataset/CellBench/10x_5cl/10x_5cl_data.csv",
    "CelSeq2_5cl_data" = "/mnt/volume1/qyc/data/Intra-dataset/CellBench/CelSeq2_5cl/CelSeq2_5cl_data.csv",
    "Filtered_Muraro_HumanPancreas_data" = "/mnt/volume1/qyc/data/Intra-dataset/Pancreatic_data/Muraro/Filtered_Muraro_HumanPancreas_data.csv",
    "Filtered_Segerstolpe_HumanPancreas_data" = "/mnt/volume1/qyc/data/Intra-dataset/Pancreatic_data/Segerstolpe/Filtered_Segerstolpe_HumanPancreas_data.csv",
    "Filtered_mouse_allen_brain_data" = "/mnt/volume1/qyc/data/Intra-dataset/AMB/Filtered_mouse_allen_brain_data.csv"
  )
  
  if (sample %in% names(data_paths)) {
    # Use readLines to count lines and subtract 1 for header
    lines <- readLines(data_paths[[sample]])
    return(length(lines) - 1)  # subtract 1 for header
  }
  return(NA)
}

In [4]:
# Get all samples
samples <- list.dirs(file.path(result_dir, tools[1]), full.names = FALSE, recursive = FALSE)

In [5]:
# Create data frame for plotting
plot_data <- expand.grid(tool = tools, sample = samples)
print("Initial plot_data dimensions:")
print(dim(plot_data))

# Debug time_data
time_data <- Map(read_time_data, plot_data$tool, plot_data$sample)
print("Number of time_data entries:")
print(length(time_data))
print("First few time_data entries:")
print(head(time_data))

# Debug mean and sd extraction
plot_data$time_mean <- sapply(time_data, function(x) x$mean)
plot_data$time_sd <- sapply(time_data, function(x) x$sd)
print("Number of non-NA time_mean values:")
print(sum(!is.na(plot_data$time_mean)))

# Debug cell numbers
plot_data$cell_number <- sapply(plot_data$sample, get_cell_number)
print("Number of non-NA cell_number values:")
print(sum(!is.na(plot_data$cell_number)))

# Remove rows with NA values
plot_data <- plot_data[!is.na(plot_data$time_mean) & !is.na(plot_data$cell_number), ]
print("Final plot_data dimensions:")
print(dim(plot_data))

# Print first few rows of final data
print("First few rows of final data:")
print(head(plot_data))

[1] "Initial plot_data dimensions:"
[1] 40  2
[1] "Number of time_data entries:"
[1] 40
[1] "First few time_data entries:"
[[1]]
[[1]]$mean
[1] 1.266581

[[1]]$sd
[1] 0.0215766


[[2]]
[[2]]$mean
[1] 174.2074

[[2]]$sd
[1] 1.233607


[[3]]
[[3]]$mean
[1] 83.9404

[[3]]$sd
[1] 1.875673


[[4]]
[[4]]$mean
[1] 2.973113

[[4]]$sd
[1] 0.04285689


[[5]]
[[5]]$mean
[1] 1.457265

[[5]]$sd
[1] 0.01587113


[[6]]
[[6]]$mean
[1] 16.3081

[[6]]$sd
[1] 1.837477


[1] "Number of non-NA time_mean values:"
[1] 40
[1] "Number of non-NA cell_number values:"
[1] 40
[1] "Final plot_data dimensions:"
[1] 40  5
[1] "First few rows of final data:"
           tool       sample  time_mean    time_sd cell_number
1            RF 10x_5cl_data   1.266581 0.02157660       23154
2          scVI 10x_5cl_data 174.207435 1.23360724       23154
3 singleCellNet 10x_5cl_data  83.940402 1.87567313       23154
4       SingleR 10x_5cl_data   2.973113 0.04285689       23154
5           SVM 10x_5cl_data   1.457265 0.01587113 

In [6]:
# scale 8:6
options(repr.plot.width=8, repr.plot.height=6)

# Create the line plot
p <- ggplot(plot_data[!plot_data$sample %in% c("Filtered_mouse_allen_brain_data","CL_pbmc1"), ], aes(x = cell_number, y = time_mean, color = tool)) +
  geom_line() +
  geom_point() +
  geom_errorbar(aes(ymin = time_mean - time_sd, ymax = time_mean + time_sd), width = 0.05) +
  scale_x_log10(breaks = c(2000, 3000, 5000, 10000, 20000, 30000)) +
  scale_y_log10() +
  scale_color_brewer(palette = "Set1") +
  labs(x = "Number of Cells (log scale)",
       y = "Time (seconds, log scale)",
       color = "Tool") +
  theme_minimal() +
  theme(legend.position = "bottom",
        text = element_text(size = 16),
        axis.title = element_text(size = 16),
        legend.text = element_text(size = 16),
        panel.grid = element_blank(),  # 移除所有网格线
        axis.line = element_line(color = "black"),  # 添加坐标轴线
        panel.border = element_blank(),
        axis.ticks = element_line(color = "black"),  # 添加刻度线
        axis.ticks.length = unit(0.2, "cm"))  # 设置刻度线长度

# Save plot to PDF
pdf("runtime_comparison.pdf", width = 8, height = 6)
print(p)
dev.off()

pdf 
  2

In [70]:
# 定义评估指标计算函数
calculate_metrics <- function(true_labels, pred_labels) {
  # 计算混淆矩阵
  conf <- table(true_labels, pred_labels)
  
  # 计算每个类别的F1分数
  unique_true <- unique(true_labels)
  unique_pred <- unique(pred_labels)
  unique_all <- unique(c(unique_true, unique_pred))
  
  # 处理未标记的预测
  pred_lab_clean <- gsub('Node..', 'Node', pred_labels)
  conf_F1 <- table(true_labels, pred_lab_clean, 
                  exclude = c('unassigned', 'Unassigned', 'Unknown', 'rand', 'Node', 'ambiguous', 'unknown'))
  
  # 计算F1分数
  F1 <- vector()
  sum_acc <- 0
  
  for (i in 1:length(unique_true)) {
    findLabel = colnames(conf_F1) == rownames(conf_F1)[i]
    if(sum(findLabel)) {
      prec <- conf_F1[i,findLabel] / colSums(conf_F1)[findLabel]
      rec <- conf_F1[i,findLabel] / rowSums(conf_F1)[i]
      if (prec == 0 || rec == 0) {
        F1[i] = 0
      } else {
        F1[i] <- (2*prec*rec) / (prec + rec)
      }
      sum_acc <- sum_acc + conf_F1[i,findLabel]
    } else {
      F1[i] = 0
    }
  }
  
  # 计算其他指标
  med_F1 <- median(F1)
  total <- length(pred_labels)
  num_unlab <- sum(pred_labels == 'unassigned') + 
               sum(pred_labels == 'Unassigned') + 
               sum(pred_labels == 'rand') + 
               sum(pred_labels == 'Unknown') + 
               sum(pred_labels == 'unknown') + 
               sum(pred_labels == 'Node') + 
               sum(pred_labels == 'ambiguous')
  per_unlab <- num_unlab / total
  acc <- sum_acc/sum(conf_F1)
  
  return(list(MedF1 = med_F1, Acc = acc, PercUnl = per_unlab))
}

In [71]:
# 创建评估结果矩阵
create_evaluation_matrix <- function() {
  # 获取所有工具和样本
  tools <- c("RF", "scVI", "singleCellNet", "SingleR", "SVM")
  result_dir <- "/mnt/volume1/qyc/auto-annot/results/"
  
  # 获取所有样本
  samples <- list.dirs(file.path(result_dir, tools[1]), full.names = FALSE, recursive = FALSE)
  
  # 创建结果数据框
  eval_results <- data.frame()
  
  # 对每个工具和样本计算指标
  for (tool in tools) {
    for (sample in samples) {
      # 读取预测标签和真实标签
      pred_file <- file.path(result_dir, tool, sample, paste0(tool, "_Pred_Labels.csv"))
      true_file <- file.path(result_dir, tool, sample, paste0(tool, "_True_Labels.csv"))
      
      if (file.exists(pred_file) && file.exists(true_file)) {
        pred_labels <- unlist(read.csv(pred_file))
        true_labels <- unlist(read.csv(true_file))
        
        # 计算指标
        metrics <- calculate_metrics(true_labels, pred_labels)
        
        # 添加到结果数据框
        new_row <- data.frame(
          tool = tool,
          sample = sample,
          MedF1 = metrics$MedF1,
          Accuracy = metrics$Acc,
          PercUnl = metrics$PercUnl
        )
        eval_results <- rbind(eval_results, new_row)
      }
    }
  }
  
  return(eval_results)
}

# 生成评估结果矩阵
eval_matrix <- create_evaluation_matrix()

# 显示结果
print(eval_matrix)

            tool                                  sample     MedF1  Accuracy
1             RF                            10x_5cl_data 1.0000000 1.0000000
2             RF                             10Xv2_pbmc1 0.8400040 0.8655524
3             RF                             10Xv3_pbmc1 0.8345773 0.8596242
4             RF                        CelSeq2_5cl_data 1.0000000 0.9982456
5             RF                                CL_pbmc1 0.8521669 0.8667105
6             RF         Filtered_mouse_allen_brain_data 0.9799303 0.9827694
7             RF      Filtered_Muraro_HumanPancreas_data 0.9646942 0.9712128
8             RF Filtered_Segerstolpe_HumanPancreas_data 0.9875425 0.9748934
9           scVI                            10x_5cl_data 1.0000000 1.0000000
10          scVI                             10Xv2_pbmc1 0.8909863 0.9160836
11          scVI                             10Xv3_pbmc1 0.9022683 0.8889284
12          scVI                        CelSeq2_5cl_data 1.0000000 1.0000000

In [123]:
options(repr.plot.width=12, repr.plot.height=10)

library(ggplot2)
library(dplyr)

# 假设 eval_matrix 有这些列：tool, sample, MedF1, Accuracy

# 长格式转换，方便画图（可选）
library(tidyr)
library(viridis)

long_data <- eval_matrix %>%
  pivot_longer(cols = c(MedF1, Accuracy),
               names_to = "Metric", values_to = "Value")

# 按指标拆分数据
data_medf1 <- long_data %>% filter(Metric == "MedF1")
data_acc <- long_data %>% filter(Metric == "Accuracy")

# Step 1: 按 MedF1 计算 tool 的均值
tool_order <- data_medf1 %>%
  group_by(tool) %>%
  summarize(mean_value = mean(Value)) %>%
  arrange(desc(mean_value)) %>%  # 从高到低排序
  pull(tool)

# Step 2: 设置 tool 的 factor 顺序
data_medf1$tool <- factor(data_medf1$tool, levels = tool_order)

# Step 3: 同样处理 Accuracy 数据
tool_order_acc <- data_acc %>%
  group_by(tool) %>%
  summarize(mean_value = mean(Value)) %>%
  arrange(desc(mean_value)) %>%
  pull(tool)

data_acc$tool <- factor(data_acc$tool, levels = tool_order_acc)

# 画 MedF1 热图
p_medf1 <- ggplot(data_medf1, aes(x = tool, y = sample, fill = Value)) +
  geom_tile(color = "white") +
  geom_text(aes(label = sprintf("%.2f", Value)), color = "white", size = 6) +
  scale_fill_viridis(option = "D", direction = 1, name = "Value")+
  theme_minimal() +
  labs(title = "", fill = "MedF1") +
  theme(
    axis.text.x = element_text(angle = 0, hjust = 0.5, size = 20),
    axis.text.y = element_text(size = 20),
    axis.title.x = element_text(size = 20),
    axis.title.y = element_text(size = 20),
    legend.title = element_text(size = 20),
    legend.text = element_text(size = 20),
    plot.title = element_text(size = 18, face = "bold")
  )

# 画 Accuracy 热图
p_acc <- ggplot(data_acc, aes(x = tool, y = sample, fill = Value)) +
  geom_tile(color = "white") +
  geom_text(aes(label = sprintf("%.2f", Value)), color = "white", size = 6) +
  scale_fill_viridis(option = "D", direction = 1, name = "Value")+
  theme_minimal() +
  labs(title = "", fill = "Accuracy") +
  theme(
    axis.text.x = element_text(angle = 0, hjust = 0.5, size = 20),
    axis.text.y = element_text(size = 20),
    axis.title.x = element_text(size = 20),
    axis.title.y = element_text(size = 20),
    legend.title = element_text(size = 20),
    legend.text = element_text(size = 20),
    plot.title = element_text(size = 18, face = "bold")
  )

# Save plots as PDF
pdf("medf1_heatmap.pdf", width = 14, height = 10)
print(p_medf1)
dev.off()

pdf("accuracy_heatmap.pdf", width = 14, height = 10)
print(p_acc)
dev.off()


pdf 
  2

pdf 
  2

In [128]:
pbmc_10x_v2_path <- "/mnt/volume1/qyc/data/Inter-dataset/PbmcBench/10Xv2/10Xv2_pbmc1.csv"
pbmc_10x_v2_dir <- dirname(pbmc_10x_v2_path)
pbmc_10x_v3_path <- "/mnt/volume1/qyc/data/Inter-dataset/PbmcBench/10Xv3/10Xv3_pbmc1.csv"
pbmc_10x_v3_dir <- dirname(pbmc_10x_v3_path)
pbmc_cel_seq_path <- "/mnt/volume1/qyc/data/Inter-dataset/PbmcBench/CEL-Seq/CL_pbmc1.csv"
pbmc_cel_seq_dir <- dirname(pbmc_cel_seq_path)
lung_cancer_10x_path <- "/mnt/volume1/qyc/data/Intra-dataset/CellBench/10x_5cl/10x_5cl_data.csv"
lung_cancer_10x_dir <- dirname(lung_cancer_10x_path)
lung_cancer_cel_seq_path <- "/mnt/volume1/qyc/data/Intra-dataset/CellBench/CelSeq2_5cl/CelSeq2_5cl_data.csv"
lung_cancer_cel_seq_dir <- dirname(lung_cancer_cel_seq_path)
pancreas_cel_seq_path <- "/mnt/volume1/qyc/data/Intra-dataset/Pancreatic_data/Muraro/Filtered_Muraro_HumanPancreas_data.csv"
pancreas_cel_seq_dir <- dirname(pancreas_cel_seq_path)
pancreas_smart_seq_path <- "/mnt/volume1/qyc/data/Intra-dataset/Pancreatic_data/Segerstolpe/Filtered_Segerstolpe_HumanPancreas_data.csv"
pancreas_smart_seq_dir <- dirname(pancreas_smart_seq_path)
brain_allen_10x_path <- "/mnt/volume1/qyc/data/Intra-dataset/AMB/Filtered_mouse_allen_brain_data.csv"
brain_allen_10x_dir <- dirname(brain_allen_10x_path)

results_dir <- "/mnt/volume1/qyc/auto-annot/results"

sample_list <- list(
  pbmc_10x_v2_path,
  pbmc_10x_v3_path,
  pbmc_cel_seq_path,
  lung_cancer_10x_path,
  lung_cancer_cel_seq_path,
  pancreas_cel_seq_path,
  pancreas_smart_seq_path,
  brain_allen_10x_path
)

dir_list <- list(
  pbmc_10x_v2_dir,
  pbmc_10x_v3_dir,
  pbmc_cel_seq_dir,
  lung_cancer_10x_dir,
  lung_cancer_cel_seq_dir,
  pancreas_cel_seq_dir,
  pancreas_smart_seq_dir,
  brain_allen_10x_dir
)

In [149]:
library(ggplot2)
library(viridis)
library(patchwork)

visualize_cell_proportions <- function(labels_path, title_text) {
  labels <- read.csv(labels_path, header = FALSE)
  cell_counts <- table(labels$V1[-1])  # 跳过第一行
  cell_props <- prop.table(cell_counts) * 100
  plot_data <- data.frame(cell_type = names(cell_props), proportion = as.numeric(cell_props))
  plot_data <- plot_data[order(plot_data$proportion, decreasing = TRUE), ]
  
  ggplot(plot_data, aes(x = reorder(cell_type, -proportion), y = proportion, fill = cell_type)) +
    geom_bar(stat = "identity", show.legend = FALSE) +
    scale_fill_viridis_d(option = "C") +
    labs(
      x = NULL,  # 去掉 x轴标题
      y = "Proportion (%)",
      title = title_text
    ) +
    theme_classic() +
    theme(
    axis.title.x = element_blank(),         # 去掉 X轴的标题“Cell Type”
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10),  # 保留刻度文字（cell types）
    axis.ticks.x = element_line(),          # 保留刻度线
    axis.text.y = element_text(size = 8),
    axis.title.y = element_text(size = 10),
    plot.title = element_text(size = 12, hjust = 0.5)
    )
}

plots <- list()

for (i in seq_along(sample_list)) {
  sample_path <- sample_list[[i]]
  dir_path <- dir_list[[i]]
  labels_path <- file.path(dir_path, "Labels.csv")
  if (!file.exists(labels_path)) {
    labels_path <- paste0(tools::file_path_sans_ext(sample_path), "Labels.csv")
  }
  sample_name <- basename(sample_path)
  plots[[i]] <- visualize_cell_proportions(labels_path, title_text = sample_name)
}

# 拼图为2行4列
combined_plot <- wrap_plots(plots, ncol = 4)

# 保存成PDF
output_dir <- "/mnt/volume1/qyc/auto-annot/pipelines/"
if (!dir.exists(output_dir)) dir.create(output_dir, recursive = TRUE)

ggsave(
  filename = file.path(output_dir, "celltype_proportions_combined.pdf"),
  plot = combined_plot,
  width = 20, height = 10, dpi = 300
)